In [ ]:
import pandas as pd

from sklearn.manifold import TSNE

import plotly.express as px

from jupyter_dash import JupyterDash

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, ALL

# Read data

In [ ]:
df = pd.read_csv(snakemake.input.fname)
df.head()

# Create Dashboard

In [ ]:
def do_tsne(df):
    df_sub = df.filter(regex='^(?!target__)')  # remove response
    target = df.filter(regex='^target__').squeeze()
    
    tsne = TSNE(n_components=2, random_state=0)
    proj = tsne.fit_transform(df_sub)
    
    return pd.DataFrame(proj, columns=['TSNE_0', 'TSNE_1'], index=target)

In [ ]:
def load_dataset(dataset):
    return pd.read_csv(f'../../resources/{dataset}.csv')

In [ ]:
# plotting
def update_figure():
    return px.bar(
        df, x='model', y='accuracy',
        facet_col='dataset',
        barmode='group', title='Overall accuracy'
    )

# create app
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Auto-ML Benchmark results'),
    dcc.Graph(figure=update_figure()),
    html.Hr(),
    dcc.Dropdown(
        id='dataset-selector',
        options=[  # TODO: dynamically set these
            {'label': 'test_dataset', 'value': 'test_dataset'},
            {'label': 'iris', 'value': 'iris'}
        ],
        value='iris'
    ),
    html.Div(id='covariate-container'),
    dcc.Graph(id='projection-container'),
    html.Div(id='prediction-container')
])

# plots with callbacks
@app.callback(
    Output('covariate-container', 'children'),
    [Input('dataset-selector', 'value')]
)
def _(dataset):
    df = load_dataset(dataset)
    
    children = []
    for col in df.columns:
        if col.startswith('target__'):
            continue
        
        children.append(dcc.Input(
            id={'type': 'covariate', 'name': col},
            placeholder=col,
            debounce=True
        ))
    
    return children

@app.callback(
    Output('projection-container', 'figure'),
    [
        Input('dataset-selector', 'value'),
        Input({'type': 'covariate', 'name': ALL}, 'value')
    ]
)
def _(dataset, *covariates):
    df = load_dataset(dataset)
    
    if len(covariates[0]) > 0 and None not in covariates[0]:
        cov_columns = df.filter(regex='^(?!target__)').columns
        target_column = df.filter(regex='^target__').columns[0]
        
        df = df.append({
            target_column: 'MANUAL',
            **dict(zip(cov_columns, covariates[0]))
        }, ignore_index=True)
    
    df_proj = do_tsne(df)
    
    return px.scatter(
        df_proj, x='TSNE_0', y='TSNE_1',
        color=df_proj.index,
        title='t-SNE projection'
    )

@app.callback(
    Output('prediction-container', 'children'),
    [
        Input('dataset-selector', 'value'),
        Input({'type': 'covariate', 'name': ALL}, 'value')
    ]
)
def _(dataset, *covariates):
    children = []
    
    return f'{dataset} - {covariates}'

# run app
app.run_server(mode='jupyterlab')